In [51]:
!pip install flask_cors

In [52]:
import pandas as pd
from flask import  Flask, request, jsonify, render_template
from flask_cors import CORS, cross_origin
import json

In [53]:
app = Flask('Nuesta API')
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

In [54]:
df = pd.read_csv('./Data/PVR_DAM_all.csv')
dam = df[df['punto_tipo'] == 'Intersercción']
pvr = df[df['punto_tipo'] != 'Intersercción']
dam['x'] = dam['x'].str.replace(',','.',)
dam['y'] = dam['y'].str.replace(',','.')

In [55]:
dam.sample(10)

,ID,radio_censal,punto_tipo,poblacion,hogares,desocupado,0-14,65_mas,inicial,primario,...,hacinamiento,sin_agua,sin_cloaca,densidad_comercial,microzonificacion,macrozonificacion,x,y,index,clasificacion
1311,I1579,500070508.0,Intersercción,734.0,265.0,19.0,104.0,151.0,24.0,52.0,...,0.0,0.0,0.0,18.0,Residencial urbano,Urbano,-68.85286367,-32.88929224,1652,0.0
1182,I1407,500070107.0,Intersercción,704.0,212.0,25.0,91.0,142.0,10.0,131.0,...,0.0,0.0,0.0,238.0,Central,Altamente urbano,-68.83928812,-32.88495644,1480,0.0
1038,I1239,500070608.0,Intersercción,949.0,349.0,16.0,165.0,115.0,37.0,106.0,...,3.0,0.0,1.0,17.0,Residencial urbano,Urbano,-68.85713344,-32.87914417,1312,1.0
111,I140,500071104.0,Intersercción,796.0,294.0,35.0,127.0,164.0,25.0,124.0,...,0.0,0.0,0.0,5.0,Residencial urbano,Urbano,-68.8491116,-32.86458146,213,1.0
263,I321,500071102.0,Intersercción,878.0,309.0,27.0,123.0,192.0,20.0,115.0,...,0.0,0.0,2.0,8.0,Residencial urbano,Urbano,-68.85480293,-32.86663504,394,1.0
1602,I1971,500070801.0,Intersercción,562.0,164.0,17.0,106.0,53.0,21.0,66.0,...,0.0,0.0,2.0,18.0,Residencial urbano,Urbano,-68.85156412,-32.89954289,2044,0.0
491,I576,500070201.0,Intersercción,1065.0,365.0,37.0,215.0,150.0,32.0,217.0,...,2.0,4.0,1.0,25.0,Residencial urbano,Urbano,-68.83242851,-32.86727432,649,1.0
1195,I1423,500070111.0,Intersercción,349.0,152.0,9.0,51.0,61.0,12.0,31.0,...,0.0,0.0,0.0,137.0,Central,Altamente urbano,-68.84347659,-32.88534948,1496,0.0
1168,I1391,500070109.0,Intersercción,634.0,202.0,18.0,87.0,97.0,16.0,102.0,...,0.0,0.0,0.0,43.0,Comercial,Altamente urbano,-68.84839402,-32.88443582,1464,0.0
180,I219,500071103.0,Intersercción,764.0,265.0,28.0,129.0,146.0,24.0,125.0,...,0.0,0.0,1.0,4.0,Residencial urbano,Urbano,-68.85275329,-32.86555963,292,1.0


In [56]:
def find_closest(df, x, y):
    df['x'] = pd.to_numeric(df['x'])
    df['y'] = pd.to_numeric(df['y'])
    dist = (df['x'] - x).abs() + (df['y'] - y).abs()
    return df.loc[dist.idxmin()]

In [57]:
@app.route('/predict',methods=['POST'])
@cross_origin()
def predict():
    data = request.get_json(force=True)
    predict = find_closest(dam, data['lng'], data['lat'])
    return predict.to_json()


In [58]:
@app.route('/damero',methods=['GET'])
@cross_origin()
def damero():
    return dam.to_json(orient = 'records')

In [59]:
@app.route('/pvr',methods=['GET'])
@cross_origin()
def pvrendpoint():
    return pvr.to_json(orient = 'records')

In [60]:
app.run(host='0.0.0.0', port=5009)

 * Serving Flask app "Nuesta API" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.68:5009/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Sep/2022 18:37:26] "GET /damero HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2022 18:37:28] "GET /damero HTTP/1.1" 200 -
